# 💡 MosaicPi_Guide: **"Linear Algebra in MosaicPi"**

> All code and examples are shared to help researchers, students, and engineers understand the reasoning behind DDDA — and to make it easy to apply dimensional analysis to your own data.  
> This notebook serves as an entry-level guide for teaching, validating physical models, and enabling domain-specific knowledge engineering through data-driven dimensional reasoning.

---

## 🎯 What You'll Learn

**MosaicPi中的线性代数应用**

This notebook introduces the **fundamental concepts of linear algebra** that are widely used in applied mathematics, data science, physics, and engineering. The focus is on building an intuitive understanding alongside practical computations using NumPy.

1. **物理模型，隐函数，流形**  
   Understand why we reduce variables and how dimensional consistency enables model generalization.

2. **变量组合**  
   Encode physical units of input quantities using base units and build the D-matrix.

3. **变量组合评估**  
   Discover dimensionless groups by solving linear algebraic equations on the D-matrix.

4. **显式化策略可视化**  
   Learn to assess whether derived groups make physical and computational sense.

5. **不确定性定量化**  
   Set the stage for further steps in the DDDA pipeline including Pi-group selection, uncertainty quantification, and regime detection.

---

## 👤 Author

- **Name**: Jiashun Pang  
- **Created**: August 2025  
- **Affiliation**: MosaicPi, open research notebook  
- **Notebook Focus**:  
  A hands-on exploration of dimensional analysis — from aggregated raw quantities to symbolic Pi-group discovery and preparation for downstream DDDA tasks.

---

📌 *This notebook is designed to be accessible for learners new to dimensional analysis, while also laying the foundation for advanced applications in the full MosaicPi pipeline.*

---

# 4. Singular Value Decomposition (SVD)

---
1. **Decomposition Form**

   * General form:

     $$
     A = U \Sigma V^T
     $$

     where

     * \$U\$: left singular vectors (orthogonal).
     * \$\Sigma\$: diagonal matrix of singular values (non-negative).
     * \$V\$: right singular vectors (orthogonal).

2. **Geometric Interpretation**

   * Any linear transformation can be expressed as:

     * **Rotation (V)** → **Stretching/Compression (\$\Sigma\$)** → **Rotation (U)**.
   * Provides insight into how a matrix acts on space.

3. **Singular Values and Stability**

   * Largest singular value: maximum stretching factor.
   * Smallest singular value: closeness to singularity.
   * \$\sigma\_{\min} \approx 0\$ → matrix nearly singular.

4. **Condition Number**

   * Defined as

     $$
     \kappa(A) = \frac{\sigma_{\max}}{\sigma_{\min}}
     $$
   * Measures numerical stability and sensitivity:

     * \$\kappa \approx 1\$: well-conditioned (stable).
     * \$\kappa \gg 1\$: ill-conditioned (unstable, sensitive to perturbations).

5. **Applications in Implicit Functions**

   * **Local Solvability**: non-zero smallest singular value indicates local invertibility.
   * **Variable Partitioning**: guides selection of \$x\$ / \$y\$ blocks in implicit–explicit decomposition.
   * **Bifurcation Detection**: \$\sigma\_{\min} \to 0\$ signals possible loss of uniqueness or transition.

---


## 3.1 SVD — Decomposition Form

### (1) 基本定义与矩阵形状

设 $A\in\mathbb{R}^{m\times n}$，则存在**奇异值分解**：

$$
A = U\,\Sigma\,V^{T},
$$

其中

* $U\in\mathbb{R}^{m\times m}$：**左奇异向量**组成的正交矩阵（$U^{T}U=I_m$）。
* $V\in\mathbb{R}^{n\times n}$：**右奇异向量**组成的正交矩阵（$V^{T}V=I_n$）。
* $\Sigma\in\mathbb{R}^{m\times n}$：**非负对角**矩阵

  $$
  \Sigma=\begin{bmatrix}
  \sigma_1 & & & \\
  & \sigma_2 & & \\
  & & \ddots & \\
  & & & \sigma_r \\
  & & & & \mathbf{0}
  \end{bmatrix},\quad
  \sigma_1\ge \sigma_2 \ge \cdots \ge \sigma_r \ge 0,\quad r=\min(m,n).
  $$

**在奇异值分解里：**

* **奇异值**就是你看到的 $\sigma_1, \sigma_2, \ldots, \sigma_r$。
* 它们存放在对角矩阵 $\Sigma$ 的对角线上，其余元素为零。
* **排列方式**通常约定为：

  $$
  \sigma_1 \geq \sigma_2 \geq \cdots \geq \sigma_r \geq 0,
  $$

  这样最大的奇异值在左上角，最小的（可能为 0）靠右下。

📌 直观理解：

* $\sigma_1$：矩阵 $A$ 作用时的**最大伸缩因子**（空间里某个方向被拉伸得最多）。
* $\sigma_r$：矩阵作用时的**最小伸缩因子**，如果它接近 0，说明 $A$ 在某些方向上几乎把向量压扁了 → 接近奇异/不可逆。

### (2) “薄/经济”与“完整”SVD

* **完整（full）SVD**：如上所述，$U$ 是 $m\times m$，$V$ 是 $n\times n$，$\Sigma$ 是 $m\times n$（带零填充）。
* **薄/经济（thin / economy）SVD**：取 $r=\operatorname{rank}(A)$ 或取 $r=\min(m,n)$ 的“瘦形”版本：

  $$
  A = U_r\,\Sigma_r\,V_r^{T},
  $$

  其中 $U_r\in\mathbb{R}^{m\times r}$、$\Sigma_r\in\mathbb{R}^{r\times r}$、$V_r\in\mathbb{R}^{n\times r}$。多数数值库默认或可选返回此版本（存储友好，运算更快）。



---

# 手动求解 SVD 的通用步骤（适用于任意 $m\times n$ 矩阵）

设 $A\in\mathbb{R}^{m\times n}$。

**Step 1｜构造对称正定矩阵**

* 计算 $A^{T}A\in\mathbb{R}^{n\times n}$（或 $AA^{T}\in\mathbb{R}^{m\times m}$）。
* 这是一个对称半正定矩阵，特征值全为非负。

**Step 2｜求特征分解**

* 解特征值问题 $(A^{T}A)v_i=\lambda_i v_i$，得到 $\lambda_i\ge 0$ 及单位化的特征向量 $v_i$。
* 把特征值按降序排序：$\lambda_1\ge\lambda_2\ge\cdots\ge\lambda_r\ge 0$。

**Step 3｜得到奇异值与右奇异向量**

* 奇异值：$\sigma_i=\sqrt{\lambda_i}$（降序排列）。
* 右奇异向量：$v_i$（上一步的单位特征向量）。
* 将 $v_i$ 作为列拼成 $V=[v_1,\dots,v_n]$（满维）或 $V_r=[v_1,\dots,v_r]$（薄 SVD）。

**Step 4｜得到左奇异向量**

* 对于 $\sigma_i>0$，定义

  $$
  u_i=\frac{1}{\sigma_i}A v_i,
  $$

  并单位化（理论上已是单位向量，数值上可再正规化一次）。
* 若 $m>r$（矩形或秩亏），用正交补（如 Gram–Schmidt）把 $\{u_1,\dots,u_r\}$ 补齐为 $\{u_1,\dots,u_m\}$。
* 拼成 $U=[u_1,\dots,u_m]$（或薄型 $U_r=[u_1,\dots,u_r]$）。

**Step 5｜组装 $\Sigma$ 并验证**

* 令 $\Sigma=\operatorname{diag}(\sigma_1,\dots,\sigma_r)$（薄型），或把其放入 $m\times n$ 的对角块（完整 SVD）。
* 检查：

  $$
  U^{T}U=I,\quad V^{T}V=I,\quad A\stackrel{?}{=}U\Sigma V^{T}.
  $$

> 要点：
> • **顺序一致**：$\sigma_i$ 的排序要与 $u_i,v_i$ 的列次序一致。
> • **零奇异值**：若 $\sigma_i=0$，则 $u_i$ 不能用 $A v_i/\sigma_i$ 定义，需要从 $\mathcal{N}(A^{T})$（左零空间）中取正交基补齐。

---

## 例子：手算一个 2×2 的 SVD

取

$$
A=\begin{bmatrix}
0 & 1\\
2 & 0
\end{bmatrix}.
$$

**① 计算 $A^{T}A$：**

$$
A^{T}A=
\begin{bmatrix}
0 & 2\\
1 & 0
\end{bmatrix}
\begin{bmatrix}
0 & 1\\
2 & 0
\end{bmatrix}
=
\begin{bmatrix}
4 & 0\\
0 & 1
\end{bmatrix}.
$$

**② 求特征分解：**

* 特征值显然是 $\lambda_1=4,\ \lambda_2=1$。
* 其对应单位特征向量可取

  $$
  v_1=\begin{bmatrix}1\\0\end{bmatrix},\quad
  v_2=\begin{bmatrix}0\\1\end{bmatrix}.
  $$

**③ 得到奇异值与右奇异向量：**

$$
\sigma_1=\sqrt{4}=2,\quad \sigma_2=\sqrt{1}=1;\quad
V=[v_1\ v_2]=I_2.
$$

**④ 计算左奇异向量：**

$$
u_1=\frac{1}{\sigma_1}A v_1
=\frac{1}{2}\begin{bmatrix}0\\2\end{bmatrix}
=\begin{bmatrix}0\\1\end{bmatrix},\qquad
u_2=\frac{1}{\sigma_2}A v_2
=\begin{bmatrix}1\\0\end{bmatrix}.
$$

因此

$$
U=\begin{bmatrix}
0 & 1\\
1 & 0
\end{bmatrix},\quad
\Sigma=\begin{bmatrix}
2 & 0\\
0 & 1
\end{bmatrix},\quad
V^{T}=I_2.
$$

**⑤ 验证分解：**

$$
U\Sigma V^{T}
=
\begin{bmatrix}
0 & 1\\
1 & 0
\end{bmatrix}
\begin{bmatrix}
2 & 0\\
0 & 1
\end{bmatrix}
=
\begin{bmatrix}
0 & 1\\
2 & 0
\end{bmatrix}
=A.
$$

且 $U^{T}U=I,\ V^{T}V=I$，完全符合 SVD 定义。

---

## 2×2 矩阵的一般闭式公式（便于手算）

对一般 $A=\begin{bmatrix}a&b\\ c&d\end{bmatrix}$：

* 计算

  $$
  A^{T}A=
  \begin{bmatrix}
  a^2+c^2 & ab+cd\\
  ab+cd & b^2+d^2
  \end{bmatrix}.
  $$
* 令

  $$
  \tau=\operatorname{tr}(A^{T}A)=a^2+b^2+c^2+d^2,\qquad
  \Delta=\det(A^{T}A)=(ad-bc)^2.
  $$
* 两个特征值（闭式）：

  $$
  \lambda_{1,2}=\frac{\tau\pm\sqrt{\tau^2-4\Delta}}{2}.
  $$
* 奇异值：

  $$
  \sigma_{1,2}=\sqrt{\lambda_{1,2}}\quad(\sigma_1\ge\sigma_2\ge 0).
  $$
* 对应右奇异向量 $v_i$ 由 $(A^{T}A-\lambda_i I)v_i=0$ 解得并单位化；
  左奇异向量 $u_i=\dfrac{1}{\sigma_i}A v_i$（若 $\sigma_i>0$）。

> 小贴士：当 $ad-bc=0$ 时，$\sigma_2=0$，矩阵秩为 1；此时第二个 $u_i$ 需用正交补来补齐。

---

## 矩形矩阵时怎么“补齐”？

* 若 $A\in\mathbb{R}^{m\times n},\ m>n$：用上面步骤得到 $r\le n$ 个 $u_i$。再从 $\mathcal{N}(A^{T})$（解 $A^{T}x=0$）中选出 $m-r$ 个正交单位向量，和前面的 $u_i$ 一起组成 $U\in\mathbb{R}^{m\times m}$。
* 若 $n>m$：类似，从 $\mathcal{N}(A)$ 中补齐 $V$ 的列。

---

## 快速检查清单（你在纸上/白板手算时）

1. 写出 $A^{T}A$；
2. 解 $\det(A^{T}A-\lambda I)=0$ 得 $\lambda_i$ 并排序；
3. $\sigma_i=\sqrt{\lambda_i}$；
4. 解特征向量 $v_i$ 并单位化；
5. $u_i=Av_i/\sigma_i$（$\sigma_i=0$ 时改用正交补）；
6. 组装 $U,\Sigma,V$，验证 $A=U\Sigma V^{T}$。


### (3) 唯一性与不唯一性

* **奇异值 $\{\sigma_i\}$**：**唯一**（按降序排列，非负）。
* **奇异向量**：在**不重复**奇异值处，方向仅差一个符号（或复数情形下一个单位相位因子）。
  若某个奇异值有重数 $>1$，对应的奇异向量张成的子空间是唯一的，但**基的选择不唯一**（可在该子空间里做任意正交（酉）变换）。

### (4) 秩与四个基本子空间（Range/Null）

记 $r=\operatorname{rank}(A)=\#\{i:\sigma_i>0\}$。

* **列空间 / 值域**：$\mathcal{R}(A)=\operatorname{span}\{u_1,\ldots,u_r\}$（$U$ 的前 $r$ 列）。
* **行空间**：$\mathcal{R}(A^{T})=\operatorname{span}\{v_1,\ldots,v_r\}$（$V$ 的前 $r$ 列）。
* **零空间**：$\mathcal{N}(A)=\operatorname{span}\{v_{r+1},\ldots,v_{n}\}$（与零奇异值对应的 $V$ 的列）。
* **左零空间**：$\mathcal{N}(A^{T})=\operatorname{span}\{u_{r+1},\ldots,u_{m}\}$（与零奇异值对应的 $U$ 的列）。

> 这四个子空间两两正交，给出线性代数的“基本定理”结构化视图，后续做变量划分、可解性分析时非常有用。

### (5) 与对称特征分解的关系

$$
A^{T}A = V\,\Sigma^{2}\,V^{T},\qquad
AA^{T} = U\,\Sigma^{2}\,U^{T}.
$$

因此 $\sigma_i^2$ 是 $A^{T}A$ 与 $AA^{T}$ 的非零特征值；$v_i$ 是 $A^{T}A$ 的特征向量，$u_i$ 是 $AA^{T}$ 的特征向量。

### (6) 与范数、行列式、条件数的关系

* **谱范数**：$\|A\|_2=\sigma_{\max}=\sigma_1$。
* **Frobenius 范数**：$\|A\|_F=\big(\sum_{i}\sigma_i^2\big)^{1/2}$。
* **行列式（方阵）**：若 $A\in\mathbb{R}^{n\times n}$，则

  $$
  |\det A|=\prod_{i=1}^{n}\sigma_i
  \quad(\text{因为 }\det U,\det V\in\{\pm1\}).
  $$
* **2-范数条件数（满秩时）**：

  $$
  \kappa_2(A)=\frac{\sigma_{\max}}{\sigma_{\min}},\quad
  \text{若 }\sigma_{\min}=0\text{ 则 }\kappa_2(A)=\infty.
  $$

### (7) 伪逆与最小二乘

* **Moore–Penrose 伪逆**：

  $$
  A^{+}=V\,\Sigma^{+}\,U^{T},\quad
  \Sigma^{+}=\operatorname{diag}\big(\sigma_1^{-1},\ldots,\sigma_r^{-1}\big).
  $$
* **最小二乘解**（不适定时取最小范数解）：

  $$
  x^{\star}=\arg\min_x\|Ax-b\|_2
  =A^{+}b=V\,\Sigma^{+}\,U^{T}b.
  $$

### (8) 最优低秩近似（Eckart–Young–Mirsky）

取前 $k$ 个奇异对 $(\sigma_i,u_i,v_i)$，得

$$
A_k=\sum_{i=1}^{k}\sigma_i\,u_i v_i^{T}
$$

是 $A$ 在 $\|\cdot\|_2$ 与 $\|\cdot\|_F$ 下的**最佳秩-$k$** 近似；误差为
$\|A-A_k\|_2=\sigma_{k+1}$，
$\|A-A_k\|_F=\big(\sum_{i>k}\sigma_i^2\big)^{1/2}$。

### (9) 数值实现与实践提示

* 常用接口：`numpy.linalg.svd(A, full_matrices=False)` 返回 $U\in\mathbb{R}^{m\times r}$、`s`（奇异值向量）、`Vt\in\mathbb{R}^{r\times n}`。
* 退化/近退化时（$\sigma_{\min}$ 很小），数值误差会放大；可用**截断 SVD**（丢弃过小奇异值）稳化伪逆与回归。

---